In [5]:
import tensorflow as tf
import numpy as np
import cv2  # Import OpenCV

def load_tflite_model(tflite_file):
    """Loads a TensorFlow Lite model."""
    interpreter = tf.lite.Interpreter(model_path=tflite_file)
    interpreter.allocate_tensors()
    return interpreter

def preprocess_image(image_path, input_size):
    """
    Reads, resizes, and preprocesses an image for TFLite inference.
    """
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
    img = cv2.resize(img, input_size)
    img = img.astype(np.float32) / 255.0  # Normalize
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

def run_inference(interpreter, image):
    """Runs inference on a TFLite model."""
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    interpreter.set_tensor(input_details[0]['index'], image)
    interpreter.invoke()
    output_data = [interpreter.get_tensor(detail['index']) for detail in output_details]
    return output_data

# Example usage:
# tflite_file = "yolov9.tflite"
image_path = "/Users/silunikeerthiratne/Documents/IoT/inference/images/8.png"  # Replace with your image path

interpreter = load_tflite_model("/Users/silunikeerthiratne/Documents/IoT/inference/best-136_float16.tflite")

input_details = interpreter.get_input_details()
input_size = input_details[0]['shape'][1:3]  # Get input height and width

preprocessed_image = preprocess_image(image_path, input_size)
output_data = run_inference(interpreter, preprocessed_image)

print(output_data)  # Process the output data as needed


/Users/silunikeerthiratne/Documents/IoT/Code/myenvIot/lib/python3.11/site-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


[array([[[9.02402960e-03, 2.01367587e-02, 2.11668015e-02, ...,
         9.29805875e-01, 9.49218333e-01, 9.63228762e-01],
        [1.88839715e-02, 2.30635032e-02, 2.06848364e-02, ...,
         9.60851610e-01, 9.56777573e-01, 9.60256696e-01],
        [2.31977738e-02, 4.61424664e-02, 4.78895232e-02, ...,
         1.37574494e-01, 1.00626707e-01, 7.45378733e-02],
        ...,
        [7.34375863e-06, 1.41843557e-05, 1.29567898e-05, ...,
         1.75613925e-04, 1.93664309e-04, 1.50466250e-04],
        [5.79053903e-06, 9.91659817e-06, 6.66892629e-06, ...,
         1.02133876e-04, 2.16766522e-04, 1.90570077e-04],
        [8.78977698e-06, 1.55106409e-05, 1.33332296e-05, ...,
         4.16680487e-05, 8.00659400e-05, 5.57757812e-05]]], dtype=float32)]


In [4]:
import tensorflow as tf

# Path to the SavedModel directory
saved_model_dir = ""  # Replace with your actual path
# Path where the TFLite model will be saved
tflite_model_path = "yolov9.tflite"

# Convert the SavedModel to TFLite
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)

# Optimization settings (optional, but recommended)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16] # if you want to use float16 quantization

tflite_model = converter.convert()

# Save the TFLite model
with open(tflite_model_path, "wb") as f:
    f.write(tflite_model)

print(f"TFLite model saved to {tflite_model_path}")


FileNotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for variables/variables
 You may be trying to load on a different device from the computational device. Consider setting the `experimental_io_device` option in `tf.saved_model.LoadOptions` to the io_device such as '/job:localhost'.